In [1]:
import re
import os
from pathlib import Path
from typing import Union
from warnings import simplefilter
from datetime import date
from shutil import disk_usage
from math import floor
from multiprocessing import Pool
from multiprocessing import cpu_count
from zipfile import ZipFile

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
test = Path(r'C:\Users\wonhyeong\downloads\20211020_10-Q_edgar_data_1456802_0001477932-21-007463.txt') #ixbrl


In [2]:
def clean_html(file: Path) -> str:
    docu_end = '</DOCUMENT'
    try:
        with open(file, 'r+', encoding='utf-8') as f:
            # read and rewrite
            text = f.read()
            found = text.find(docu_end)
            if found == -1:
                found = text.find(docu_end.lower())
            if found == -1:
                raise ValueError('No document end tag found')
            text = text[:text.find(docu_end)]
            f.close()
    except:
        try:
            f.close()
        except:
            pass
        return file, None, False
    return file, text, True

def make_folder(file: Union[str, Path]) -> Path:
    name = '_'.join([str(file.name),'cleaned'])
    cleaned = file.parent / name
    done= cleaned / 'done'
    errored = cleaned / 'errored'
    cleaned.mkdir(exist_ok=True)
    done.mkdir(exist_ok=True)
    errored.mkdir(exist_ok=True)
    return done, errored

def save_on_success(file: Path, text: str, done: Path) -> None:
    print(f'Saving {file.name}')
    with open(file, 'w', encoding='utf-8') as f:
        f.write(text)
        f.close()
    file.rename(done / file.name)
    

def save_on_failure(file: Path, errored_dir: Path) -> None:
    print(f'Error in {file.name}')
    file.rename(errored_dir / file.name)

def get_free_space():
    """return free space of disk as GB"""
    _,_,free = disk_usage('C:\\')
    result = round(free / 1024**3, 2)
    print(result)
    return result

def unzip_file(zip_file: Union[str, Path]) -> None:
    """Unzip a file to the same directory."""
    with ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(zip_file.parent)
    return None

In [3]:
qtr = Path(r'C:\Users\wonhyeong\workings\data\10X\QTR4')

if __name__ == '__main__':
    files = list(qtr.iterdir())
    zip_list = [x for x in files if x.is_file() and x.suffix == '.zip']
    done, errored = make_folder(qtr)
    free_space = get_free_space()
    if free_space > 500:
        unzip_file(zip_list[0])
        zip_list = zip_list[1:]
        zip_list[0].unlink()
    with Pool(cpu_count() * 2) as p:
        result = p.imap(clean_html, files, chunksize=100) 
        for file, text, success in result:
            if success:
                save_on_success(file, text, done)
            else:
                save_on_failure(file, errored)
    print('Done')

25.75


In [10]:
a

<generator object Pool.imap.<locals>.<genexpr> at 0x000001AA5066ADD0>